In [1]:
import logging as lg
from pathlib import Path
from keras.utils import to_categorical
from keras.datasets import mnist
import numpy as np
%matplotlib inline

Using TensorFlow backend.


In [2]:
verbose=True
BATCH_SIZE=100
VALID_RATE=0.3

RESULT_DIR=Path(f"result/multi_mnist_{BATCH_SIZE}_{VALID_RATE}")
if verbose:
    lg.getLogger().setLevel(lg.INFO)
    lg.info("Verbose mode activated")

INFO:root:Verbose mode activated


In [3]:
from tools.models import KerasRegressorInterface

In [15]:
from sklearn.model_selection import train_test_split
(x_train_val, y_train_val), (X_test, y_test) = mnist.load_data()
X_train, X_valid, y_train, y_valid = train_test_split(x_train_val, y_train_val, test_size=VALID_RATE)

In [29]:
from keras.preprocessing.image import ImageDataGenerator
class MultipleMnistGenerator:
    def __init__(self):
        self.img_gen= ImageDataGenerator(rescale=1./255)
         
    def flow(self,images,labels,batch_size=1):
        images=self.preprocess(images)
        img_g1=self.img_gen.flow(images,labels,batch_size=batch_size)
        img_g2=self.img_gen.flow(images,labels,batch_size=batch_size)
        while True:
            img1,label1=next(img_g1)
            img2,label2=next(img_g2)
            label=label1*label2         
            yield [img1,img2],label
            
    def preprocess(self,images):
        images=images.reshape(images.shape[0], 28,28,1)
        return images

In [26]:
class MyGenerator:
    """今回は使わない."""
    def __init__(self):
        pass
    
    def flow(self,x,batch_size):
        i=0
        while True:
            if (i+1)*batch_size<=x.shape[0]:
                yield x[i * (batch_size):(i+1)*batch_size]
                i+=1
            else:
                yield x[i * (batch_size):]
                i=0

In [30]:
gen=MultipleMnistGenerator()
g_train=gen.flow(X_train,y_train,BATCH_SIZE)
g_valid=gen.flow(X_valid,y_valid)
g_test=gen.flow(X_test,y_test)

In [31]:
inputs,labels=next(g_train)
inputs[0].shape,labels.shape

((100, 28, 28, 1), (100,))